# SQL

## Описание проекта

### Цель исследования

Проанализировать базу данных и сформировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Подключение к БД и вывод первых строк таблиц

In [1]:
# загрузка библиотек
import pandas as pd
from sqlalchemy import create_engine

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# установка параметров
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохранение коннектора
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# первые 5 строк таблицы с книгами
query = '''
        SELECT * 
          FROM books
         LIMIT 5; 
        '''
pd.io.sql.read_sql(query, con=engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# первые 5 строк таблицы с авторами
query = '''
        SELECT * 
          FROM authors
         LIMIT 5; 
        '''
pd.io.sql.read_sql(query, con=engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# первые 5 строк таблицы с издателями
query = '''
        SELECT * 
          FROM publishers
         LIMIT 5; 
        '''
pd.io.sql.read_sql(query, con=engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# первые 5 строк таблицы с рейтингами
query = '''
        SELECT * 
          FROM ratings
         LIMIT 5; 
        '''
pd.io.sql.read_sql(query, con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# первые 5 строк таблицы с обзорами
query = '''
        SELECT * 
          FROM reviews
         LIMIT 5; 
        '''
pd.io.sql.read_sql(query, con=engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Выполнение запросов

#### Задание №1

Посчитайте, сколько книг вышло после 1 января 2000 года

#### Решение

In [7]:
query = '''
        SELECT COUNT(book_id) 
          FROM books
         WHERE publication_date > '2000-01-01'; 
        '''
pd.io.sql.read_sql(query, con=engine)

,count
0,819


#### Вывод

Всего после 1 января 2000 года вышло 819 книг, что составляет в среднем 37 книг в год или 3 книги в месяц.

#### Задание №2

Для каждой книги посчитайте количество обзоров и среднюю оценку

#### Решение

In [14]:
query = '''
        WITH a AS(
        
                  SELECT b.title,
                         COUNT(rev.review_id) AS cnt_reviews
                    FROM books AS b
                    LEFT JOIN reviews AS rev ON b.book_id=rev.book_id
                   GROUP BY b.title
                   ORDER BY COUNT(rev.review_id) DESC
                  ),
        
        b AS(
        
                  SELECT b.title,
                         AVG(rat.rating) AS avg_rating
                    FROM books AS b
                    LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
                   GROUP BY b.title
                   ORDER BY AVG(rat.rating) DESC
                  )
        
        SELECT a.title,
               a.cnt_reviews,
               ROUND(b.avg_rating, 2) AS avg_rating
          FROM a
          JOIN b ON a.title=b.title
         ORDER BY cnt_reviews DESC,
                  avg_rating DESC;
        '''
pd.io.sql.read_sql(query, con=engine)

,title,cnt_reviews,avg_rating
0,Memoirs of a Geisha,8,4.11
1,Twilight (Twilight #1),7,3.66
2,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
4,The Book Thief,6,4.26
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.00
995,Essential Tales and Poems,0,4.00
996,Leonardo's Notebooks,0,4.00
997,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


#### Вывод

Всего представлено 999 книг, из них наибольшее количество обзоров - 8 штук - сделано на книгу "Мемуары Гейши" и средним рейтингом 4.1 по 5-бальной шкале. Второй по количеству обзоров является книга "Сумерки" у которой 7 обзоров, а средний рейтинг составляет 3.66. Также в первую пятерку попадают книги о Гарри Поттере: 2 и 3 серия книги по 6 обзоров на каждую, и книга "Книжный вор" у которой тоже 6 обзоров.

#### Задание №3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

#### Решение

In [9]:
query = '''
        SELECT publisher
          FROM (
          
               SELECT pub.publisher,
                      COUNT(b.book_id) AS cnt_books
                 FROM books AS b
                 JOIN publishers AS pub ON b.publisher_id=pub.publisher_id
                WHERE num_pages > 50
                GROUP BY pub.publisher
                ORDER BY COUNT(b.book_id) DESC
                LIMIT 1) AS pb;
           
        '''
pd.io.sql.read_sql(query, con=engine)

,publisher
0,Penguin Books


#### Вывод

Наибольшее число книг толще 50 страниц выпустило издательство "Penguin Books".

#### Задание №4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

#### Решение

In [15]:
query = '''
        WITH tab AS(

        SELECT author,
               AVG(rating) AS avg_rating
          FROM (
                SELECT b.book_id,
                       author,
                       COUNT(rating_id) AS cnt_rating
                  FROM ratings AS r
                  JOIN books AS b ON b.book_id=r.book_id
                  JOIN authors AS a ON b.author_id=a.author_id
                 GROUP BY b.book_id, author
                HAVING COUNT(rating_id) >= 50 
                 ORDER BY cnt_rating DESC) AS a
                        
          LEFT JOIN (
                SELECT book_id,
                       rating
                  FROM ratings) AS b ON a.book_id=b.book_id 
                  
         GROUP BY author
         ORDER BY avg_rating DESC)
         
        SELECT author, 
               ROUND(avg_rating, 2) AS avg_rating
          FROM tab
         LIMIT 1; 
        '''
pd.io.sql.read_sql(query, con=engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


#### Вывод

Самая высокая средняя оценка книг у автора J.K.Rowling. В расчете приняли участие только книги, в которых 50 и более оценок.

#### Задание №5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

#### Решение

In [16]:
query = '''
        SELECT ROUND(AVG(b.cnt_reviews), 2) AS avg_reviews
          FROM (
                SELECT username,
                       COUNT(rating)
                  FROM ratings
                 GROUP BY username
                HAVING COUNT(rating) > 50 
                 ORDER BY COUNT(rating) DESC
                ) AS a
        
          LEFT JOIN (
          
                SELECT username,
                       COUNT(review_id) AS cnt_reviews
                  FROM reviews 
                 GROUP BY username
                ) AS b ON a.username=b.username 
        '''
pd.io.sql.read_sql(query, con=engine)

,avg_reviews
0,24.33


#### Вывод

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 24 штуки.

## Общий вывод

- Всего после 1 января 2000 года вышло 819 книг, что составляет в среднем 37 книг в год или 3 книги в месяц;
- Всего представлено 999 книг, из них наибольшее количество обзоров - 8 штук - сделано на книгу "Мемуары Гейши" и средним рейтингом 4.1 по 5-бальной шкале. Второй по количеству обзоров является книга "Сумерки" у которой 7 обзоров, а средний рейтинг составляет 3.66. Также в первую пятерку попадают книги о Гарри Поттере: 2 и 3 серия книги по 6 обзоров на каждую, и книга "Книжный вор" у которой тоже 6 обзоров;
- Наибольшее число книг толще 50 страниц выпустило издательство "Penguin Books";
- Самая высокая средняя оценка книг у автора J.K.Rowling. В расчете приняли участие только книги, в которых 50 и более оценок;
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, составляет 24 штуки.